## 目的
- 学習データをどう分割するかを学ぶ

## 1. いままでのパターン
- `train_test_split()` を使ってデータセットを学習データとテストデータに分割
- 学習データに対して ` fit()` を呼び出してモデルを構築
- テストデータに対して  `score()` で評価


In [1]:
# Warningを非表示にする
import warnings

warnings.filterwarnings('ignore')

In [2]:
# データの準備
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, random_state=0)

In [3]:
# LogsticRegressionで学習
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

# ...

logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [4]:
# モデルの評価
logreg.score(X_test, y_test)

0.868421052631579

## 2. データ分割方法を考える

## 2.1 学習データ == テストデータ
- やってはいけない
- 過学習がおきてしまう

## 2.2 ホールドアウト
- いままでの方法はコレ
- でも分割の割合はどうすれば良いと思う？
    - 古典的なのはあるところで分割
    - `train_test_split()`を使っていた
- 学習誤差がでるけど、データを増やせば改善される


## 2.3 交差検証
- データの分割を何度も行い、複数のモデルを訓練する
    - k分割交差検証(k-fold cross-validation)
    - 1つの分割したセットを Fold と呼ぶ
    - 標準はデータの先頭から分割する
-  最終的に k個 の精度が手に入る

In [5]:
from sklearn.model_selection import cross_val_score

iris = load_iris()
logreg = LogisticRegression()

scores = cross_val_score(logreg, iris.data,  iris.target)

# デフォルトでは 3分割 なので 3つの精度 が手に入る
print(scores)

[0.96078431 0.92156863 0.95833333]


In [6]:
# 分割数を変えてみる(k=5)
scores = cross_val_score(logreg, iris.data,  iris.target, cv=5)

print(scores)

[1.         0.96666667 0.93333333 0.9        1.        ]


In [7]:
# 精度は平均でまとめるのが一般的
scores.mean()

0.9600000000000002

#### 結論: このモデルは平均でおよそ96%の割合で正しいだろう

#### 考察: 90% 〜 100% のばらつきがある件について == 分割間での精度のばらつきが比較的大きい
- 可能性1: このモデルは訓練に用いられた特定の分野に強く依存している？
- 可能性2: 単純にデータ数が少ない？

### 交差検証のメリット/デメリット
- ただランダムに分割するよりもいいことがいくつかある

### メリット
#### 1.  モデルはデータセットのすべてのサンプルに対して良い汎化性能を示さなければ交差検証スコアは高くできない
- ランダム分割で精度が高くなりすぎてしまうパターン
    - クラス分類が難しいデータが学習データに、クラス分類が簡単なデータがテストデータに。
- ランダム分割で精度が低くなりすぎてしまうパターン
    - クラス分類が簡単なデータが学習データに、クラス分類が難しいデータがテストデータに。

#### 2. データをより効率的に使える
- 通常の `train_test_split()`
    - 25%をテストデータにしておしまい
- 交差検証(k=5の場合)
    - それぞれの回で80%で訓練し、20%でテストする

### デメリット
- 計算コストがかかる！

## 2.4 層化しないと危ない
- クラスバランスの考慮が必要

In [8]:
from sklearn.model_selection import KFold

# 層化されていない3分割の悲劇を確認する
kfold = KFold(n_splits=3)

# 学習データはある1つのクラスのみ テストデータは別の2クラス なので全く正解しない。まったく学習していない
cross_val_score(logreg, iris.data, iris.target, cv=kfold)

array([0., 0., 0.])

In [9]:
# クラスごとにまとまっているから！！ ← さっきはTrainTestSplitしていたからセーフ
print(iris.target)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [10]:
# 作戦: シャッフルしてから分割する
kfold = KFold(n_splits=3, shuffle=True, random_state=0)

cross_val_score(logreg, iris.data, iris.target, cv=kfold)

array([0.9 , 0.96, 0.96])

### 2.5 1つ抜き交差検証(Leave-One-Out)
- 計算コストは高いが、小さいデータセットに対してはよりよい推定が可能になる


In [11]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)

print("Number of Iterations:", len(scores))
print("Mean Accuracy:", scores.mean())

Number of Iterations: 150
Mean Accuracy: 0.9533333333333334


## 2.6 シャッフル分割交差検証
- 復元抽出を使ったパターン

In [12]:
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=10)

cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)

array([0.94666667, 0.90666667, 0.92      , 0.86666667, 0.97333333,
       0.92      , 0.92      , 0.92      , 0.97333333, 0.94666667])

## 補足
`cross_validate()` という関数を使うと学習時間なども取得できる

In [13]:
from sklearn.model_selection import cross_validate

iris = load_iris()
logreg = LogisticRegression()

scores = cross_validate(logreg, iris.data, iris.target)

for key, value in scores.items():
    print(f"{key}: {value.mean():.4f}")

fit_time: 0.0005
score_time: 0.0002
test_score: 0.9469
